In [8]:
from pptx import Presentation

In [9]:
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

In [10]:
import json

with open("content.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(data)


{'slides': [{'id': 1, 'type': 'title', 'title': 'Online vs. Offline Learning: Your Journey', 'bullets': [], 'notes': "This presentation critically examines the benefits and challenges of both online and offline learning environments. We aim to help today's students make informed decisions about their educational path.", 'image_keywords': ['student learning', 'education path', 'digital vs traditional study'], 'design_aesthetics': {'layout': 'full-bleed-image', 'title_position': 'center', 'text_style': 'minimal', 'image_style': 'background'}, 'image_path': 'temp_images\\temp_image_slide1.jpg'}, {'id': 2, 'type': 'content', 'title': "Today's Agenda: Charting Your Learning Course", 'bullets': ['Understanding the evolving educational landscape.', 'Strengths of online learning.', 'Value of traditional classrooms.', 'Key differences & common challenges.', 'Strategies for choosing your best path.'], 'notes': "We'll explore both educational models, highlight their unique attributes, and discuss

In [11]:
prs = Presentation()

All other slides

In [12]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.enum.shapes import MSO_SHAPE
from pptx.dml.color import RGBColor
import os


In [13]:
def add_overlay(slide, left, top, width, height, color_overlay):
    """Adds a black/brand color semi-transparent rectangle behind text."""
    rect = slide.shapes.add_shape(
        MSO_SHAPE.RECTANGLE, left, top, width, height
    )
    fill = rect.fill
    fill.solid()
    fill.fore_color.rgb = RGBColor(
        color_overlay["r"],
        color_overlay["g"],
        color_overlay["b"]
    )
    fill.transparency = color_overlay["alpha"]
    rect.line.fill.background()


def center_position(prs, box_w, box_h):
    left = (prs.slide_width - box_w) / 2
    top = (prs.slide_height - box_h) / 2
    return left, top


In [ ]:
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT, MSO_AUTO_SIZE
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_SHAPE
from PIL import Image, ImageDraw
import os
from colorsys import hex_to_rgb

# Extract theme config from data or use defaults
def get_theme_config(data):
    if isinstance(data, dict) and "theme_config" in data:
        config = data["theme_config"]
        colors = config.get("colors", {})
        contrast = config.get("contrast_factor", 1.0)
        
        # Convert hex to RGB
        def hex_to_rgb_tuple(hex_color):
            hex_color = hex_color.lstrip('#')
            return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
        
        # Apply contrast factor to RGB
        def apply_contrast(rgb, factor):
            return tuple(int(min(255, c * factor)) for c in rgb)
        
        theme = {
            "primary": apply_contrast(hex_to_rgb_tuple(colors.get("primary", "#1F1F28")), contrast),
            "secondary": apply_contrast(hex_to_rgb_tuple(colors.get("secondary", "#6032B6")), contrast),
            "accent": apply_contrast(hex_to_rgb_tuple(colors.get("accent", "#00D9FF")), contrast),
            "text_primary": hex_to_rgb_tuple(colors.get("text_primary", "#FFFFFF")),
            "text_secondary": hex_to_rgb_tuple(colors.get("text_secondary", "#B0B0B0")),
            "background": apply_contrast(hex_to_rgb_tuple(colors.get("background", "#0F0F15")), contrast)
        }
        return theme
    else:
        # Fallback to default theme
        return {
            "primary": (31, 31, 40),
            "secondary": (96, 50, 150),
            "accent": (0, 217, 255),
            "text_primary": (255, 255, 255),
            "text_secondary": (176, 176, 176),
            "background": (15, 15, 21)
        }

theme = get_theme_config(data)

# Style constants
MARGIN = Inches(0.6)
CONTENT_W = prs.slide_width - 2 * MARGIN
CONTENT_H = prs.slide_height - 2 * MARGIN
MAX_IMG_W = int(CONTENT_W * 0.42)
MAX_IMG_H = int(CONTENT_H - Inches(1))

# Sanitize text
def sanitize_text(text):
    if not isinstance(text, str):
        return text
    cleaned = text.replace('***', ' ').replace('**', ' ').replace('*', ' ')
    return ' '.join(cleaned.split())

# Set dark/themed slide background
def set_slide_background(slide, color):
    bg = slide.background
    fill = bg.fill
    fill.solid()
    fill.fore_color.rgb = RGBColor(*color)

# Add rounded content panel with theme accent border
def add_content_panel(slide, primary_color, accent_color):
    panel = slide.shapes.add_shape(
        MSO_SHAPE.ROUNDED_RECTANGLE,
        MARGIN,
        MARGIN,
        CONTENT_W,
        CONTENT_H
    )
    panel.fill.solid()
    panel.fill.fore_color.rgb = RGBColor(*primary_color)
    line = panel.line
    line.fill.solid()
    line.color.rgb = RGBColor(*accent_color)
    line.width = Pt(2.4)
    return panel

# Fit image with PIL and add border/shadow effect
def fit_image_size(img_path, max_w, max_h):
    with Image.open(img_path) as im:
        w_px, h_px = im.size
        dpi = im.info.get('dpi', (72, 72))[0] or 72
        w_in = w_px / dpi
        h_in = h_px / dpi
        img_w = Inches(w_in)
        img_h = Inches(h_in)
        scale = min(max_w / img_w, max_h / img_h, 1)
        return int(img_w * scale), int(img_h * scale)

# Add subtle image border/shadow using PIL
def place_image_with_border(slide, img_path, left, top, max_w, max_h, accent_color):
    w, h = fit_image_size(img_path, max_w, max_h)
    
    # Load and add border to image
    with Image.open(img_path) as im_orig:
        # Create new image with border (4px accent color border)
        border_size = 8
        border_color = accent_color
        im_with_border = Image.new('RGB', (im_orig.width + 2*border_size, im_orig.height + 2*border_size), border_color)
        im_with_border.paste(im_orig, (border_size, border_size))
        
        # Save temp image
        temp_img_path = img_path.replace('.jpg', '_bordered.jpg').replace('.png', '_bordered.png')
        im_with_border.save(temp_img_path, quality=95)
        
        # Place image
        slide.shapes.add_picture(temp_img_path, left, top, width=w, height=h)
        
        # Clean up temp file
        try:
            os.remove(temp_img_path)
        except:
            pass

# Enhanced textbox with better font rendering
def create_text_box(slide, left, top, width, height, text, size=Pt(20), bold=False, color=(255,255,255), align=PP_PARAGRAPH_ALIGNMENT.LEFT):
    text = sanitize_text(text)
    tf = slide.shapes.add_textbox(left, top, width, height).text_frame
    tf.word_wrap = True
    try:
        tf.margin_top = int(Inches(0.08))
        tf.margin_left = int(Inches(0.08))
        tf.margin_right = int(Inches(0.08))
        tf.margin_bottom = int(Inches(0.08))
    except:
        pass
    tf.clear()
    p = tf.paragraphs[0]
    p.text = text
    p.font.size = size
    p.font.bold = bold
    p.font.color.rgb = RGBColor(*color)
    p.font.name = 'Calibri'
    p.alignment = align
    return tf

# Enhanced bulleted list with better styling
def create_bulleted_list(slide, left, top, width, height, items, size=Pt(20), color=(255,255,255), accent_color=(0,217,255)):
    items = [sanitize_text(i) for i in items if i]
    tf = slide.shapes.add_textbox(left, top, width, height).text_frame
    tf.word_wrap = True
    try:
        tf.margin_top = int(Inches(0.04))
        tf.margin_left = int(Inches(0.06))
        tf.margin_right = int(Inches(0.06))
        tf.margin_bottom = int(Inches(0.04))
    except:
        pass
    tf.clear()
    for idx, it in enumerate(items):
        if idx == 0:
            p = tf.paragraphs[0]
        else:
            p = tf.add_paragraph()
        # Use styled bullet with accent color reference
        p.text = '▸ ' + it
        p.level = 0
        p.font.size = size
        p.font.color.rgb = RGBColor(*color)
        p.font.name = 'Calibri'
        p.font.bold = False
        try:
            p.space_before = Pt(3)
            p.space_after = Pt(3)
        except:
            pass
    return tf

# Enhanced footer with theme colors
def add_footer(slide, text='', text_color=(176,176,176)):
    if not text:
        text = 'Generated with Powerpoint Generator Agent'
    w = Inches(3.5)
    h = Inches(0.3)
    left = prs.slide_width - w - MARGIN
    top = prs.slide_height - h - Inches(0.15)
    tf = slide.shapes.add_textbox(left, top, w, h).text_frame
    tf.word_wrap = False
    tf.clear()
    p = tf.paragraphs[0]
    p.text = sanitize_text(text)
    p.font.size = Pt(10)
    p.font.italic = True
    p.font.color.rgb = RGBColor(*text_color)
    p.font.name = 'Calibri'
    p.alignment = PP_PARAGRAPH_ALIGNMENT.RIGHT

# Enhanced generators with theme support

def gen_title(slide_json, prs):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    set_slide_background(slide, theme["background"])
    add_content_panel(slide, theme["primary"], theme["accent"])

    img = slide_json.get('image_path')
    if img and os.path.exists(img):
        left = MARGIN + CONTENT_W - MAX_IMG_W - Inches(0.25)
        top = MARGIN + Inches(0.6)
        place_image_with_border(slide, img, left, top, MAX_IMG_W, MAX_IMG_H, theme["accent"])

    tb_left = MARGIN + Inches(0.8)
    tb_top = MARGIN + Inches(0.9)
    tb_w = CONTENT_W - MAX_IMG_W - Inches(1.8)
    tb_h = Inches(2.3)
    create_text_box(slide, tb_left, tb_top, tb_w, tb_h, slide_json.get('title',''), size=Pt(60), bold=True, color=theme["text_primary"])

    subtitle = slide_json.get('subtitle','')
    if subtitle:
        create_text_box(slide, tb_left, tb_top + Inches(2.2), tb_w, Inches(1.0), subtitle, size=Pt(18), color=theme["accent"])

    add_footer(slide, color=theme["text_secondary"])
    return slide


def gen_full_bleed(slide_json, prs):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    set_slide_background(slide, theme["background"])
    img = slide_json.get('image_path')
    if img and os.path.exists(img):
        left = MARGIN
        top = MARGIN
        slide.shapes.add_picture(img, left, top, width=CONTENT_W, height=CONTENT_H)

    title = slide_json.get('title','')
    if title:
        box_h = Inches(1.2)
        rect = slide.shapes.add_shape(MSO_SHAPE.ROUNDED_RECTANGLE, MARGIN + Inches(0.6), MARGIN + CONTENT_H - box_h - Inches(0.25), CONTENT_W - Inches(1.2), box_h)
        rect.fill.solid()
        rect.fill.fore_color.rgb = RGBColor(*theme["secondary"])
        rect.fill.transparency = 0.15
        rect.line.fill.background()
        create_text_box(slide, MARGIN + Inches(0.75), MARGIN + CONTENT_H - box_h - Inches(0.15), CONTENT_W - Inches(1.5), box_h, title, size=Pt(34), bold=True, color=theme["text_primary"], align=PP_PARAGRAPH_ALIGNMENT.CENTER)

    add_footer(slide, color=theme["text_secondary"])
    return slide


def gen_text_left_image_right(slide_json, prs):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    set_slide_background(slide, theme["background"])
    add_content_panel(slide, theme["primary"], theme["accent"])

    img = slide_json.get('image_path')
    title = sanitize_text(slide_json.get('title',''))
    bullets = [sanitize_text(b) for b in slide_json.get('bullets', [])]

    left = MARGIN + Inches(0.8)
    top = MARGIN + Inches(0.8)
    left_w = int(CONTENT_W * 0.56)
    left_h = CONTENT_H - Inches(1.4)

    title_box_h = Inches(1.4)
    create_text_box(slide, left, top, left_w, title_box_h, title, size=Pt(38), bold=True, color=theme["accent"])

    body_top = top + title_box_h + Inches(0.15)
    if bullets:
        create_bulleted_list(slide, left, body_top, left_w, left_h - title_box_h, bullets, size=Pt(20), color=theme["text_primary"], accent_color=theme["accent"])

    if img and os.path.exists(img):
        img_left = MARGIN + left_w + Inches(0.6)
        img_top = MARGIN + Inches(0.8)
        place_image_with_border(slide, img, img_left, img_top, int(CONTENT_W - left_w - Inches(1.2)), left_h, theme["accent"])

    add_footer(slide, color=theme["text_secondary"])
    return slide


def gen_image_left_text_right(slide_json, prs):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    set_slide_background(slide, theme["background"])
    add_content_panel(slide, theme["primary"], theme["accent"])

    img = slide_json.get('image_path')
    title = sanitize_text(slide_json.get('title',''))
    bullets = [sanitize_text(b) for b in slide_json.get('bullets', [])]

    if img and os.path.exists(img):
        img_left = MARGIN + Inches(0.8)
        img_top = MARGIN + Inches(0.8)
        place_image_with_border(slide, img, img_left, img_top, MAX_IMG_W, MAX_IMG_H, theme["accent"])

    tx_left = MARGIN + MAX_IMG_W + Inches(1.2)
    tx_top = MARGIN + Inches(0.8)
    tx_w = CONTENT_W - MAX_IMG_W - Inches(1.8)
    tx_h = CONTENT_H - Inches(1.4)

    title_box_h = Inches(1.1)
    create_text_box(slide, tx_left, tx_top, tx_w, title_box_h, title, size=Pt(38), bold=True, color=theme["accent"])
    if bullets:
        create_bulleted_list(slide, tx_left, tx_top + title_box_h + Inches(0.1), tx_w, tx_h - title_box_h, bullets, size=Pt(20), color=theme["text_primary"])

    add_footer(slide, color=theme["text_secondary"])
    return slide


def gen_two_column(slide_json, prs):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    set_slide_background(slide, theme["background"])
    add_content_panel(slide, theme["primary"], theme["accent"])

    title = sanitize_text(slide_json.get('title',''))
    bullets = [sanitize_text(b) for b in slide_json.get('bullets', [])]
    left_col = bullets[:len(bullets)//2]
    right_col = bullets[len(bullets)//2:]

    create_text_box(slide, MARGIN + Inches(0.8), MARGIN + Inches(0.6), CONTENT_W, Inches(1.0), title, size=Pt(38), bold=True, color=theme["accent"])
    if left_col:
        create_bulleted_list(slide, MARGIN + Inches(0.8), MARGIN + Inches(1.6), int(CONTENT_W*0.45), CONTENT_H - Inches(1.6), left_col, size=Pt(18), color=theme["text_primary"])
    if right_col:
        create_bulleted_list(slide, MARGIN + int(CONTENT_W*0.52), MARGIN + Inches(1.6), int(CONTENT_W*0.45), CONTENT_H - Inches(1.6), right_col, size=Pt(18), color=theme["text_primary"])

    add_footer(slide, color=theme["text_secondary"])
    return slide


def gen_quote(slide_json, prs):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    set_slide_background(slide, theme["background"])
    add_content_panel(slide, theme["primary"], theme["accent"])

    quote = sanitize_text((slide_json.get('bullets') or [slide_json.get('title','')])[0])
    box_w = int(CONTENT_W * 0.9)
    box_h = Inches(1.8)
    left = MARGIN + (CONTENT_W - box_w) / 2
    top = MARGIN + (CONTENT_H - box_h) / 2
    rect = slide.shapes.add_shape(MSO_SHAPE.ROUNDED_RECTANGLE, left, top, box_w, box_h)
    rect.fill.solid()
    rect.fill.fore_color.rgb = RGBColor(*theme["secondary"])
    rect.fill.transparency = 0.12
    rect.line.color.rgb = RGBColor(*theme["accent"])
    rect.line.width = Pt(1.5)
    create_text_box(slide, left + Inches(0.25), top + Inches(0.12), box_w - Inches(0.5), box_h - Inches(0.3), '"' + quote + '"', size=Pt(28), bold=True, color=theme["accent"], align=PP_PARAGRAPH_ALIGNMENT.CENTER)

    add_footer(slide, color=theme["text_secondary"])
    return slide

# Layout mapping
LAYOUT_MAP = {
    'title': gen_title,
    'full-bleed-image': gen_full_bleed,
    'text-left-image-right': gen_text_left_image_right,
    'image-left-text-right': gen_image_left_text_right,
    'two-column': gen_two_column,
    'comparison-split': gen_two_column,
    'quote-emphasis': gen_quote
}

def render_slide(slide_json, prs):
    layout = slide_json.get('design_aesthetics', {}).get('layout', '')
    func = LAYOUT_MAP.get(layout, gen_text_left_image_right)
    return func(slide_json, prs)

# Render all slides
for slide_json in data['slides']:
    render_slide(slide_json, prs)

prs.save('final_output.pptx')
print(f"✓ Presentation saved: final_output.pptx")
print(f"✓ Theme applied: {data.get('theme_config', {}).get('theme_name', 'Default')}")
print(f"✓ Contrast level: {data.get('theme_config', {}).get('contrast', 'Normal')}")
